<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/main/notebooks/Analyse%20CH%20Compare%20Part%20Time%20Work%20Percentage%20For%20Wealthiest%20And%20Poorest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

#  Analyse CH Compare Part Time Work Percentage For Wealthiest And Poorest

*Instructions:*

1. Execute the first code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.

In [6]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.14 s (started: 2021-05-03 22:13:40 +00:00)


In [7]:
# Install required dependancies
!pip install pyspark
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.9 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [8]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 13.2 ms (started: 2021-05-03 22:14:10 +00:00)


In [9]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 7.06 ms (started: 2021-05-03 22:14:10 +00:00)


In [10]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 5.29 s (started: 2021-05-03 22:14:10 +00:00)


In [11]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 21.2 s (started: 2021-05-03 22:14:15 +00:00)


In [12]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ch'

time: 1.58 ms (started: 2021-05-03 22:14:36 +00:00)


In [13]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [14]:
# Create tempview for sparksql query
df_soc.createOrReplaceTempView('clean_ch')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, part_time_work_behavior_devices FROM clean_ch WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+-------------------------------+
|         cbg|                date|device_count|part_time_work_behavior_devices|
+------------+--------------------+------------+-------------------------------+
|170312304002|2020-02-07T00:00:...|          76|                              9|
|170313005002|2020-02-07T00:00:...|          72|                             12|
|170314208001|2020-02-07T00:00:...|         127|                             11|
|170317608034|2020-02-07T00:00:...|         114|                             11|
|170318033003|2020-02-07T00:00:...|          91|                              8|
|170318146005|2020-02-07T00:00:...|          51|                              1|
|170318182002|2020-02-07T00:00:...|          51|                              3|
|170318240053|2020-02-07T00:00:...|         123|                             26|
|170318348001|2020-02-07T00:00:...|          32|                              4|
|170438411042|2020-02-07T00:

In [15]:
# Taking substring of date for "yyyy-mm-dd"
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 80.9 ms (started: 2021-05-03 22:15:10 +00:00)


In [16]:
df_soc.show()

+------------+----------+------------+-------------------------------+
|         cbg|      date|device_count|part_time_work_behavior_devices|
+------------+----------+------------+-------------------------------+
|170312304002|2020-02-07|          76|                              9|
|170313005002|2020-02-07|          72|                             12|
|170314208001|2020-02-07|         127|                             11|
|170317608034|2020-02-07|         114|                             11|
|170318033003|2020-02-07|          91|                              8|
|170318146005|2020-02-07|          51|                              1|
|170318182002|2020-02-07|          51|                              3|
|170318240053|2020-02-07|         123|                             26|
|170318348001|2020-02-07|          32|                              4|
|170438411042|2020-02-07|          57|                              7|
|170438427041|2020-02-07|          84|                             17|
|18089

In [17]:
# Add part time percentage column
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)

#df_soc.show()

time: 57.1 ms (started: 2021-05-03 22:15:11 +00:00)


In [18]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 43 ms (started: 2021-05-03 22:15:11 +00:00)




---


**Demographic data**





---



In [19]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ch.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+-------------------+
|         cbg|pop_total|poverty_percentage|     perc_whiteonly|
+------------+---------+------------------+-------------------+
|170312909001|      935|  55.8288770053476|                0.0|
|170312909002|     1129|45.261293179805136|                0.0|
|170312909003|      765|  39.7078353253652|                0.0|
|170312909004|     1075|30.325581395348838|                0.0|
|170318387002|     1016|31.003937007874015| 13.484251968503939|
|170313102001|      835|16.766467065868262|  78.20359281437126|
|170313102002|      686| 16.61807580174927| 50.583090379008745|
|170314005002|      807|40.644361833952914|                0.0|
|170314804006|      710| 1.971830985915493|  1.267605633802817|
|170314804007|      650|               0.0|                0.0|
|170314805001|     1141|  9.55302366345311|  4.206836108676599|
|170312608002|     1100| 55.45454545454545| 2.8181818181818183|
|170312609001|      836| 44.856459330143

In [20]:
# Performing inner join on mobility and demographic table using cbg
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+-------------------------------+-------------------------+------------------+
|         cbg|      date|device_count|part_time_work_behavior_devices|part_time_work_percentage|poverty_percentage|
+------------+----------+------------+-------------------------------+-------------------------+------------------+
|170312304002|2020-02-07|          76|                              9|       11.842105263157894|19.847972972972975|
|170313005002|2020-02-07|          72|                             12|       16.666666666666664|22.696011004126547|
|170314208001|2020-02-07|         127|                             11|        8.661417322834646| 34.01677539608574|
|170317608034|2020-02-07|         114|                             11|        9.649122807017543|13.651647612642906|
|170318033003|2020-02-07|          91|                              8|        8.791208791208792| 4.035608308605341|
|170318146005|2020-02-07|          51|                              1|  

In [21]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("part_time_work_percentage").withColumnRenamed('avg(part_time_work_percentage)','part_time_work_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("part_time_work_percentage").withColumnRenamed('avg(part_time_work_percentage)','part_time_work_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.part_time_work_percentage as top20_perc,
l.part_time_work_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")

time: 373 ms (started: 2021-05-03 22:15:13 +00:00)


**Load 2019 stats**

In [22]:
# Getting mean of 2019 value for part time device count
filename = '{}_social_summary_2019.csv'.format(city)
stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/filename))
stat_2019.show()
mean_2019 = stat_2019.collect()[1][6]


+-----+------------------+------------------+------------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+
|  _c0|             total|   completely_home|    part_time_work|   full_time_work|percentage_completely_home|percentage_part_time_work|percentage_full_time_work|norm_completely_home| norm_part_time_work| norm_full_time_work|
+-----+------------------+------------------+------------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+
|count|         2430227.0|         2430227.0|         2430227.0|        2430227.0|                 2430227.0|                2430227.0|                2430227.0|           2430227.0|           2430227.0|           2430227.0|
| mean| 95.04381072220825|29.833984232748627|  8.54960421392734|5.511380212630343|        30.9662652

In [23]:
# Calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 41.4 ms (started: 2021-05-03 22:15:15 +00:00)


In [24]:
# Taking bottom and top 20 percent to get the wealthiest and poorest population data
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 49.1 ms (started: 2021-05-03 22:15:15 +00:00)



**Visualizations** 



In [25]:
# Create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': ['#FF4500','#a0aab4','#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')
# 

ThemeRegistry.enable('my_theme')

time: 39.2 ms (started: 2021-05-03 22:15:15 +00:00)


In [26]:
# Define characteristics for visualisation creation
def create_chart(df) :
  chart = alt.Chart(df).transform_fold(
    fold=['Bottom 20 % Deviation', 'Top 20 % Deviation'],
    as_ = ['Poverty %','part_time_work']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = 'Date'),
      y= alt.Y('part_time_work:Q', title = 'Part Time %'),
      color='Poverty %:N'
    ).properties(width = 1000).interactive()

  rule = alt.Chart(pd.DataFrame({'Mean 2019':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='Mean 2019',
    size=alt.value(2),
    )
  
  text = alt.Chart({'values':[{ 'y': 1}]}).mark_text(
    text='Mean 2019', angle=0
    ).encode(
      y=alt.Y('y:Q'),opacity=alt.value(0.4)
    )

  return  chart + rule + text


time: 28.1 ms (started: 2021-05-03 22:15:15 +00:00)


In [27]:
# Function to filter date from dataset
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 7.38 ms (started: 2021-05-03 22:15:15 +00:00)


In [28]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,14.838739,8.705504,5.859402,-0.273834
1,2020-04-13,4.162511,5.697567,-4.816826,-3.281770
2,2020-06-24,6.133126,5.802382,-2.846211,-3.176956
3,2020-06-08,6.171660,6.228398,-2.807677,-2.750939
4,2020-09-12,4.650525,3.925098,-4.328812,-5.054239
...,...,...,...,...,...
361,2020-04-05,2.795588,3.023700,-6.183750,-5.955637
362,2020-05-01,5.769365,6.391830,-3.209973,-2.587508
363,2020-10-25,4.333854,6.272525,-4.645484,-2.706812
364,2020-12-28,7.782939,4.636885,-1.196399,-4.342452


time: 5min 6s (started: 2021-05-03 22:15:15 +00:00)


In [29]:
df = df.rename(columns={'top20_perc': 'Top 20 %', 'bottom20_perc': 'Bottom 20 %','top20_deviation': 'Top 20 % Deviation', 'bottom20_deviation': 'Bottom 20 % Deviation'})

time: 2.85 ms (started: 2021-05-03 22:20:22 +00:00)


In [30]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['Bottom 20 %', 'Top 20 %'],
    as_ = ['Poverty %','part_time_percentage']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('part_time_percentage:Q', title = 'Part Time %'), color='Poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 140 ms (started: 2021-05-03 22:20:22 +00:00)


In [31]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
#df['date'] = [datetime.datetime.date(d) for d in df['date']] 

time: 3.85 ms (started: 2021-05-03 22:20:22 +00:00)


#Filtering based on important dates

**March 17, 2020:**
Governor Pritzker announced statewide school closures beginning March 17 until March 30.












In [32]:
start_date = '2020-3-12'
end_date = '2020-4-3'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1)
chart

alt.LayerChart(...)

time: 79.4 ms (started: 2021-05-03 22:20:22 +00:00)




**April 20, 2020:**
Illinois state topped 3,000 new cases in a day for the first time. 









In [33]:
start_date = '2020-4-14'
end_date = '2020-5-1'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2)
chart
#df2

alt.LayerChart(...)

time: 71.7 ms (started: 2021-05-03 22:20:22 +00:00)



**June 3, 2020:** 
Chicago began gradually allowing businesses to open doors for outside dining and other services, though some shops remained closed following lootings in the aftermath of George Floyd's death, during phase three of its reopening.




In [34]:
start_date = '2020-6-1'
end_date = '2020-6-16'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3)
chart

alt.LayerChart(...)

time: 64.4 ms (started: 2021-05-03 22:20:22 +00:00)


**September 8, 2020:** Chicago Public Schools began its fall quarter with remote learning, with families across the city reporting a range of successes and frustrations.


In [35]:
start_date = '2020-09-1'
end_date = '2020-09-14'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4)
chart

alt.LayerChart(...)

time: 61.1 ms (started: 2021-05-03 22:20:22 +00:00)
